In [1]:
import pandas as pd
import numpy as np
import math
import utm

In [2]:
df = pd.read_excel("Households_Data.xlsx",sheet_name='Hogares')

In [3]:
dictionary=pd.read_excel("Households_Data.xlsx",sheet_name='Dictionary')

## Categorical Data

We would like to identify which columns are categorical data and which arecontinuous data, because data types are important in any machine learning algorithm.

In [4]:
dictionary

,Variable,Description,Used in Model?
0,Hogar,ID of Household,NaN
1,EXCLUIR,Exclude from estimation,NaN
2,paraValidacion,sample for validation,NaN
3,Sector,Exogenous sector (7 sectors used for exogenous...,YES (Segmentation in Exogenous Zones Model)
4,DirCoordX,UTM coordinate X,NaN
5,DirCoordY,UTM coordinate Y,NaN
6,MontoArr,Monthly dwelling rent (Chilean Peso),YES (Likelihood function)
7,IngresoHogar,Household income (Chilean Peso),NaN
8,Factor,Expansion factor of household in survey,NaN
9,AnosEstudio,Years of formal Education of head of household,NaN


From the description column, we would make an informed guess about which columns are categorical and which are continuous. 

Our guess is {Hogar, EXCLUIR, paraValidacion, Sector, CLASE, CLUSTER7, CLUSTER2} are categorical.

In [5]:
categorical_1=['Hogar', 'EXCLUIR', 'paraValidacion', 
               'Sector', 'CLASE', 'CLUSTER7', 'CLUSTER2']

The number of unique values in a column would, in general (not always), indicates whether that column is of categorical data type or continuous data type. 

In [6]:
for c in df.columns:
    print(c + " has " + str(len(df[c].unique().tolist())) + " unique value")

Hogar has 18264 unique value
EXCLUIR has 2 unique value
paraValidacion has 2 unique value
Sector has 7 unique value
DirCoordX has 16940 unique value
DirCoordY has 16930 unique value
MontoArr has 4432 unique value
IngresoHogar has 12905 unique value
Factor has 6433 unique value
AnosEstudio has 23 unique value
CLASE has 4 unique value
Sup_Prom_Constr_Hab_EOD has 452 unique value
Calid_EOD_norm_inv has 7 unique value
DensConstr_EOD has 421 unique value
Dist_est_Metro_MC_KM has 16952 unique value
Dist_salida_Autop_MC_KM has 16952 unique value
Tiempo_Com_Stgo has 98 unique value
Ingreso_Promedio_Zona_MM has 789 unique value
Acc_Comercio_tpte_pub has 631 unique value
Acc_Educacion_tpte_pub has 631 unique value
Acc_Habitacion_tpte_pub has 631 unique value
Acc_Industria_tpte_pub has 631 unique value
Acc_Servicios_tpte_pub has 631 unique value
Acc_Comercio_auto has 637 unique value
Acc_Educacion_auto has 635 unique value
Acc_Habitacion_auto has 637 unique value
Acc_Industria_auto has 636 unique

From observing the number of unique values in each column, we think {EXCLUIR, paraValidacion, Sector, CLASE, Calid_EOD_norm_inv, CLUSTER7, CLUSTER2} are categorical.

In [7]:
categorical_2=['EXCLUIR', 'paraValidacion', 'Sector', 'CLASE', 
               'Calid_EOD_norm_inv', 'CLUSTER7', 'CLUSTER2']

For columns appear in both categorical_1 and categorical_2 lists, we will take them as categorical data. For those only appear in one of the lists, we will take a closer look.

In [8]:
# in categorical_1 but not in categorical_2
set(categorical_1)-set(categorical_2)

{'Hogar'}

Hogar is the housing ID which is by definition categorical but we will omit it because ID is just a serial number which does not affect analysis.

In [9]:
# in categorical_2 but not in categorical_1
set(categorical_2)-set(categorical_1)

{'Calid_EOD_norm_inv'}

In [10]:
df['Calid_EOD_norm_inv'].unique().tolist()

[0.98, 1.0, 0.78, 0.93, 0.36, 0.55, 0.0]

It seems Calid_EOD_norm_inv is continuous. We will treat it as continuous.

In [11]:
categorical = set(categorical_1 + categorical_2)
categorical -= {'Hogar','Calid_EOD_norm_inv'}

In [12]:
categorical

{'CLASE', 'CLUSTER2', 'CLUSTER7', 'EXCLUIR', 'Sector', 'paraValidacion'}

## Feature Selection

By observing variable description, we will remove certain columns which have limited contribution to the prediction task. We will remove:

* EXCLUIR, because it seems to be used for an estimation which is not specified here, perhaps for other study purposes.
* paraValidacion, becuase it seems to be used for validation for some other studies, having little relevance here.

In [14]:
df=df.drop(['EXCLUIR', 'paraValidacion'], axis=1)

## Column Renaming

The current column names are difficult to understand, so we rename them.

In [16]:
df=df.rename(columns={'Hogar':'ID', 
                   'Sector': 'exogenous_sector', 
                   'DirCoordX' : 'UTM_East',
                   'DirCoordY' : 'UTM_North', 
                   'MontoArr' : 'monthly_rent',
                   'IngresoHogar' : 'household_income', 
                   'Factor' : 'survey_factor',
                   'AnosEstudio' : 'education_years', 
                   'CLASE' : 'type',
                   'Sup_Prom_Constr_Hab_EOD': 'built_surface',
                   'Calid_EOD_norm_inv' : 'construction_quality',
                   'DensConstr_EOD' : 'build_density', 
                   'Dist_est_Metro_MC_KM' : 'subway_dis',
                   'Dist_salida_Autop_MC_KM' : 'highway_dis', 
                   'Tiempo_Com_Stgo' : 'time_CBD',
                   'Ingreso_Promedio_Zona_MM' : 'avg_zone_income',
                   'Acc_Comercio_tpte_pub' : 'commerce_transit', 
                   'Acc_Educacion_tpte_pub' : 'eduction_transit',
                   'Acc_Habitacion_tpte_pub' : 'habitational_transit', 
                   'Acc_Industria_tpte_pub' : 'industry_transit',
                   'Acc_Servicios_tpte_pub' : 'service_transit',
                   'Acc_Comercio_auto' : 'commerce_car',
                   'Acc_Educacion_auto' : 'education_car',
                   'Acc_Habitacion_auto' : 'education_car',
                   'Acc_Habitacion_auto' : 'habitational_car',
                   'Acc_Industria_auto' : 'industry_car',
                   'Acc_Servicios_auto' : 'service_car',
                   'CLUSTER7' : 'cluster_7',
                   'CLUSTER2' : 'cluster_2'
                  })

Lastly, we need to update our categorical list.

In [18]:
# update categorical list
print("original : ")
print(categorical)
categorical = ['type', 'cluster_2', 'cluster_7', 'exogenous_sector']
print("updated : ")
print(categorical)

original : 
{'EXCLUIR', 'paraValidacion', 'CLUSTER7', 'CLASE', 'CLUSTER2', 'Sector'}
updated : 
['type', 'cluster_2', 'cluster_7', 'exogenous_sector']


## Cleaning

avg_zone_income is in Million Chilean Peso. However, monthly_rent and household_income are in Chilean Peso. We would like to standardize into Chilean Peso.

In [33]:
df['avg_zone_income'] =  df['avg_zone_income'] * 1000000

## Coordinate Data Utilization

The coordinate data given in the data set is in UTM format. It has little help because it only pinpoint the geographical location of each point. We have to quantify it into some parameters such that it can be used as a feature into any machine learning algorithm.

One way to achieve it is to get the distance between each point and the city center.

In order to get the distance, we have to rely on google map API. However, google map API does not support UTM format. So we have to obtain corresponding latitude longitude information. Luckily, python has a package named utm (https://github.com/Turbo87/utm).

### Convert UTM Format To Lat / Lon Format

We define a function to convert UTM format to Lat/Lon format.

Since the dataset does not contain zone and latitude band information for Santiage, we search online (https://www.gps-latitude-longitude.com/gps-coordinates-of-santiago-de-chile) and obtain zone to be 19 and latitude band to be 'H'.

In [21]:
def getLatLon(east, north):
    return utm.to_latlon(east, north, 19, 'H')

Again, we define a function to extract lat/lon information and add new columns to the dataset.

In [23]:
def LatLonColumns():
    res=[]
    east=df.UTM_East.tolist()
    north=df.UTM_North.tolist()
    for i in range(len(east)):
        res.append(getLatLon(east[i],north[i]))
    return res
res=LatLonColumns()
latitude=list(map(lambda x : x[0], res))
longitude=list(map(lambda x : x[1], res))

In [24]:
df['latitude']=latitude
df['longitude']=longitude

With latitude and longtitude information, we do not need UTM information any more.

In [26]:
df=df.drop(['UTM_East', 'UTM_North'], axis=1)